# Solving 3-SAT Problem with Hybrid Quantum-classical Solver 

**Author:** Xinghui Jia & Siwei Tan

**Date:** 9/4/2024

Based on paper "[HyQSAT: A Hybrid Approach for 3-SAT Problems by Integrating Quantum Annealer with CDCL][1]" (HPCA 2023)

[1]: https://ieeexplore.ieee.org/document/10071022


A propositional satisfiability (SAT) problem is to find an assignment for each variable to satisfy a given Boolean formula. A typical SAT problem is composed of multiple clauses represented as a disjunction of Boolean variables. 3-SAT problem is a special case of the SAT problem that has no more than three variables in each clause. 

3-SAT problem is a fundamental problem in various applications, such as artificial intelligence, circuit analysis, and protein structure prediction. Since the 3-SAT problem is an NP-complete problem, the time complexity of classical algorithms increases exponentially with the number of clauses and variables. We observe that the classical CDCL algorithm has the advantage of solving larger problems, while QA is more suitable for solving small but hard problems. To combine their advantages, we introduce HyQSAT in this notebook, a hybrid approach that integrates quantum annealing (QA) with the classical Conflict-Driven Clause Learning (CDCL) algorithm to enable end-to-end acceleration for solving 3-SAT problems. 


<div style="text-align:center;">
    <img src="../picture/5_1_hyqsat.png"  width="45%" height="45%">
</div>

The figure above shows the workflow of QuCT. It features a cross-iterative process. The CDCL algorithm searches for the solution. During the search, it identifies and sends the hard sub-problem to QA. QuCT designed a fast compilation flow to embed the problem and optimize the noise. It also designs multiple strategies to guide the search of CDCL using the QA results.

In [1]:
import os
os.chdir("..")
import sys
sys.path.append('..')
import logging
logging.basicConfig(level=logging.WARN)

from janusq.hyqsat import solve_by_janusct, solve_by_minisat

In [2]:
# Configure the model
file_path = "cnf_examples/UF100/uf100-01.cnf"  # input cnf flie
cpu_lim = 0  # limit the cpu time (s). 0 means infinite
mem_lim = 0  # limit the memory 0 means infinite
strictp = True  # isStrict
use_realQC = False # use real quantum computer provided by dwave, but you need to get dwave auth.

### Using HyQSAT API to Solve the Problem

We can use a QA simulator or real-world QA hardware to solve the SAT roblem. For example, here we use the simualtor.

In [3]:
# solve by HyQSAT
solve_by_janusct(file_path, verb=True, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 1,
 'conflicts': 14,
 'conflict cost': 0.22,
 'decisions': 1,
 'decisions cost': 0.03,
 'propagations': 598,
 'propagations cost': 0.731,
 'conflict literals': 97,
 'actual CPU time': 2.36728,
 'solving time': 2.369,
 'annealing time': 0.92,
 'simulation time': 9.25759,
 'quantum success number': 10,
 'quantum conflict number': 36,
 'quantum one time solve number': 0,
 'is satisfiable': True,
 'is sat': True}

Note that when using the simulator, the solving time is estimated as (CDCL time + number of QA $\times$ 120 $\mu s$).

### Comparison to MiniSAT

We compare it to the MiniSAT solver.

In [4]:
# solve by minisat
solve_by_minisat(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 3,
 'conflicts': 384,
 'conflict cost': 4.017,
 'decisions': 463,
 'decisions cost': 0.64,
 'propagations': 9130,
 'propagations cost': 17.075,
 'conflict literals': 2416,
 'actual CPU time': 0.034302,
 'solving time': 24.504,
 'annealing time': 0.0,
 'simulation time': 0.0,
 'quantum success number': 0,
 'quantum conflict number': 0,
 'quantum one time solve number': 0,
 'is satisfiable': True,
 'is sat': True}

In [5]:
all_result = {}
dir_name = 'cnf_examples/test'
for filename in os.listdir(dir_name):
    janusq_res = solve_by_janusct(os.path.join(dir_name, filename), verb=False)
    minisat_res = solve_by_minisat(os.path.join(dir_name, filename), verb=False)
    for key in janusq_res:
        if key not in all_result:
            all_result[key] = {
                'janus': 0,
                'minisat': 0
            }
        all_result[key]['janus'] += janusq_res[key]
        all_result[key]['minisat'] += minisat_res[key]

print(all_result)    

{'restarts': {'janus': 12, 'minisat': 19}, 'conflicts': {'janus': 146, 'minisat': 1383}, 'conflict cost': {'janus': 1.131, 'minisat': 7.297000000000001}, 'decisions': {'janus': 31, 'minisat': 1814}, 'decisions cost': {'janus': 0.7610000000000001, 'minisat': 1.287}, 'propagations': {'janus': 4527, 'minisat': 32252}, 'propagations cost': {'janus': 4.4159999999999995, 'minisat': 16.072}, 'conflict literals': {'janus': 985, 'minisat': 8726}, 'actual CPU time': {'janus': 23.3142, 'minisat': 0.21076300000000003}, 'solving time': {'janus': 22.531999999999996, 'minisat': 29.849}, 'annealing time': {'janus': 6.18, 'minisat': 0.0}, 'simulation time': {'janus': 45.85356000000001, 'minisat': 0.0}, 'quantum success number': {'janus': 134, 'minisat': 0}, 'quantum conflict number': {'janus': 175, 'minisat': 0}, 'quantum one time solve number': {'janus': 0, 'minisat': 0}, 'is satisfiable': {'janus': 12, 'minisat': 12}, 'is sat': {'janus': 12, 'minisat': 12}}


In [6]:
import pandas as pd

df = pd.DataFrame(all_result)
df

,restarts,conflicts,conflict cost,decisions,decisions cost,propagations,propagations cost,conflict literals,actual CPU time,solving time,annealing time,simulation time,quantum success number,quantum conflict number,quantum one time solve number,is satisfiable,is sat
janus,12,146,1.131,31,0.761,4527,4.416,985,23.314200,22.532,6.18,45.85356,134,175,0,12,12
minisat,19,1383,7.297,1814,1.287,32252,16.072,8726,0.210763,29.849,0.00,0.00000,0,0,0,12,12
